**Se procede a importar las variables de entorno y almacenarlas en variables globales**

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from datetime import datetime
load_dotenv()

True

**se definen a su vez las variables que se emplearan como parametros**

In [2]:
API_KEY = os.getenv("API_KEY")
API_URL= os.getenv("API_URL")
LANGUAGE = os.getenv("LANGUAGE","es-ES")
API_VERSION = os.getenv("API_VERSION",'3')
maximum_release_date = datetime.now().strftime("%Y-%m-%d")
minimum_release_date = "2020-10-15"

**Se realiza una request a la api de the movie database**

NOTA: puesto que la api recupera la información de manera paginada, se procedio a hacer un ciclo for para disparar la request en un total de 10 veces, y de esa manera, para  cada pagina recuperada se cargaba en el dataframe final (df)

In [3]:
num_pages = 100
df = pd.DataFrame()
for page in range(1, num_pages + 1):
    request = f'{API_URL}/{API_VERSION}/movie/now_playing?api_key={API_KEY}&language={LANGUAGE}&primary_release_date.gte={maximum_release_date}&primary_release_date.lte={minimum_release_date}&page={page}'
    response = requests.get(request)

    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])
        temp_df = pd.DataFrame(results)
        columns_to_drop = ["backdrop_path", "id", "original_language", "poster_path", "title", "video", "genre_ids"]
        temp_df.drop(columns=columns_to_drop, inplace=True)
        df = pd.concat([df, temp_df], ignore_index=True)
    else:
        print(f'Error en la solicitud de la página {page} => {response.status_code}')


**Se muestra los datos obtenidos en un dataframe**

In [4]:
df

,adult,original_title,overview,popularity,release_date,vote_average,vote_count
0,False,Five Nights at Freddy's,Un guardia de seguridad con problemas comienza...,7124.811,2023-10-25,8.475,1005
1,False,The Nun II,"En 1956 en Francia, un sacerdote es asesinado ...",2558.391,2023-09-06,6.973,1211
2,False,Saw X,"Entre los acontecimientos de 'Saw' y 'Saw II',...",2473.173,2023-09-26,7.369,624
3,False,The Exorcist: Believer,Cuando su hija Angela y su amiga Katherine mue...,2315.215,2023-10-04,6.144,344
4,False,The Equalizer 3,Desde que renunció a su vida como asesino del ...,1881.143,2023-08-30,7.370,1239
...,...,...,...,...,...,...,...
1995,False,Bloom,San Borondón es una isla mítica que aparece y ...,0.756,2023-09-30,0.000,0
1996,False,INSIDE: Narrative of Our Journey,,0.726,2023-05-03,0.000,0
1997,False,Scratch,,0.724,2023-09-29,0.000,0
1998,False,L'envoûtement,,0.718,2023-09-25,0.000,0


## Cargar los datos en un csv
Si bien los datos se pueden acceder desde la API, cabe destacar que hay cierta demorar al recuperar por iteraciones de a 20 por limite definido en el mismo servicio. De este modo se opto volcar los datos obtenidos en un csv a modo de poder trabajar más rapidamente.

In [5]:
df.to_csv('data/movies_db.csv',sep=';', index=False)

In [6]:
df = pd.read_csv('data/movies_db.csv', delimiter=';', header=0)
df.shape
df.head(3)

,adult,original_title,overview,popularity,release_date,vote_average,vote_count
0,False,Killers of the Flower Moon,Miembros de la tribu Osage de los Estados Unid...,1334.175,2023-10-18,7.633,1420
1,False,Freelance,Mason Petit (John Cena) es un soldado retirado...,1125.462,2023-01-05,6.512,304
2,False,Leave the World Behind,Las vacaciones de una familia en una casa de l...,902.926,2023-11-22,6.640,751
